In [1]:
import numpy as np
import torch
from tqdm import tqdm

In [2]:
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  164093
Total Vocab:  65


In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163993


In [37]:
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)

In [38]:
lookback = 1
print(X.shape, y.shape)

torch.Size([163993, 100, 1]) torch.Size([163993])


In [61]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=100, num_layers=2, batch_first=True)
        self.linear = nn.Linear(100, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x

In [62]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data

def my_collate(batch):

    # Preparing input sequences
    x = [item[0] for item in batch]
    x = torch.stack(x)
    # Preparing target values
    y = [item[1] for item in batch]
    y = torch.stack(y)

    return [x, y]

device = torch.device("cuda:0")

model = LSTMModel().to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
loader = tqdm(data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=128))

best_model = None
best_loss = np.inf

n_epochs = 40
for epoch in range(n_epochs):
    model.float()
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch.float().to(device))
        loss = loss_fn(y_pred.to(device), y_batch.long().to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.float().to(device))
            loss += loss_fn(y_pred.to(device), y_batch.long().to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))

torch.save([best_model, char_to_int], "single-char2.pth")

100%|██████████| 1282/1282 [00:15<00:00, 81.49it/s]


Epoch 0: Cross-entropy: 3681.1152
Epoch 1: Cross-entropy: 3450.7322
Epoch 2: Cross-entropy: 3329.1887
Epoch 3: Cross-entropy: 3247.1357
Epoch 4: Cross-entropy: 3134.8245
Epoch 5: Cross-entropy: 3060.2080
Epoch 6: Cross-entropy: 2970.8145
Epoch 7: Cross-entropy: 2897.7041
Epoch 8: Cross-entropy: 2855.8677
Epoch 9: Cross-entropy: 2803.6838
Epoch 10: Cross-entropy: 2739.0198
Epoch 11: Cross-entropy: 2679.2400
Epoch 12: Cross-entropy: 2640.1343
Epoch 13: Cross-entropy: 2603.7817
Epoch 14: Cross-entropy: 2582.1470
Epoch 15: Cross-entropy: 2522.6814
Epoch 16: Cross-entropy: 2508.6467
Epoch 17: Cross-entropy: 2460.3457
Epoch 18: Cross-entropy: 2437.6306
Epoch 19: Cross-entropy: 2409.8735
Epoch 20: Cross-entropy: 2408.4814
Epoch 21: Cross-entropy: 2365.3513
Epoch 22: Cross-entropy: 2335.8066
Epoch 23: Cross-entropy: 2308.9231
Epoch 24: Cross-entropy: 2299.7517
Epoch 25: Cross-entropy: 2276.2441
Epoch 26: Cross-entropy: 2255.6306
Epoch 27: Cross-entropy: 2231.6162
Epoch 28: Cross-entropy: 2216.

In [63]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]

In [65]:
import numpy as np
import torch
import torch.nn as nn

best_model, char_to_int = torch.load("single-char2.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())

model = LSTMModel()
model.load_state_dict(best_model)

filename = "wonderland.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = 'alice was at the beach and '
print("xd " + prompt + " dx ")
pattern = [char_to_int[c] for c in prompt]

model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x)
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

xd alice was at the beach and  dx 
Prompt: "alice was at the beach and "
the courdr was the rabbit with the white ragd in a foeng with the terps of the tertes of the tertes of throeets if in a sooe of the coulous crrious the project gutenberg liters and agdins of the teres agdinsing the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agdine the project gutenberg ligene agd